In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

/Users/cumeo/anaconda3/envs/newlogic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/cumeo/anaconda3/envs/newlogic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/cumeo/anaconda3/envs/newlogic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/cumeo/anaconda3/envs/newlogic/lib

In [2]:
from cleverhans.attacks import FastGradientMethod
from cleverhans.model import CallableModelWrapper
from cleverhans.utils import AccuracyReport
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf

In [3]:
import torch
from torchvision import datasets, transforms

In [4]:
BATCH_SIZE = 128

In [5]:
def evaluate_model(path):
    torch_model_orig = torch.load(path)
    torch_model = lambda x: torch_model_orig(x)[0] # to standard format
    with tf.Session() as sess:
        x_op = tf.placeholder(tf.float32, shape=(None, 1, 28, 28,))    
        tf_model_fn = convert_pytorch_model_to_tf(torch_model, out_dims=10)    
        cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')
        # Create an FGSM attack
        fgsm_op = FastGradientMethod(cleverhans_model, sess=sess)
        fgsm_params = {'eps': 0.3, 'clip_min': 0., 'clip_max': 1.}
        adv_x_op = fgsm_op.generate(x_op, **fgsm_params)
        adv_preds_op = tf_model_fn(adv_x_op)
        test_loader = torch.utils.data.DataLoader(
          datasets.MNIST('./mnist', train=False, transform=transforms.ToTensor()),
          batch_size=BATCH_SIZE)
        # Run an evaluation of our model against fgsm
        total = 0
        correct = 0
        for xs, ys in tqdm(test_loader):
            adv_preds = sess.run(adv_preds_op, feed_dict={x_op: xs})
            correct += (np.argmax(adv_preds, axis=1) == ys.cpu().detach().numpy()).sum()
            total += test_loader.batch_size
    acc = float(correct) / total
    print('Adv accuracy: {:.3f}'.format(acc * 100))

In [6]:
evaluate_model('output/cnn-mnist-relu.pkl')

100%|██████████| 10/10 [00:17<00:00,  1.74s/it]

Adv accuracy: 3.301


In [7]:
evaluate_model('output/cnn-mnist-relu-strictening_0.1.pkl')

100%|██████████| 10/10 [00:18<00:00,  1.86s/it]

Adv accuracy: 19.326


In [8]:
evaluate_model('output/cnn-mnist-relu-strictening_5.pkl')

100%|██████████| 10/10 [00:18<00:00,  1.84s/it]

Adv accuracy: 22.471


In [7]:
evaluate_model('output/cnn-mnist-relog.pkl')

100%|██████████| 79/79 [00:30<00:00,  2.58it/s]

Adv accuracy: 61.046


In [8]:
evaluate_model('output/cnn-mnist-relog-more-neurons.pkl')

100%|██████████| 79/79 [02:07<00:00,  1.62s/it]

Adv accuracy: 30.597


In [9]:
evaluate_model('output/cnn-mnist-relog-maxout.pkl')

100%|██████████| 79/79 [01:52<00:00,  1.42s/it]

Adv accuracy: 67.662


In [13]:
evaluate_model('output/cnn-mnist-relog-maxout-sigmoid-out.pkl')

100%|██████████| 10/10 [03:30<00:00, 21.08s/it]

Adv accuracy: 14.619


In [14]:
evaluate_model('output/cnn-mnist-relog-minmaxout-sigmoid-out.pkl')

100%|██████████| 10/10 [11:19<00:00, 67.99s/it]

Adv accuracy: 28.047


In [6]:
evaluate_model('output/cnn-mnist-relog-minmaxout_4_2-sigmoid-out-strictening_1.pkl')

100%|██████████| 79/79 [06:44<00:00,  5.12s/it]

Adv accuracy: 24.812
